In [1]:
"""Display random patches for Amazon deforestation prediction."""

import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import h5py
from pathlib import Path

# Add src to path
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

from utils.amazon_plots import plot_prediction_patch

def load_predictions_from_h5(h5_file_path):
    """Load predictions from HDF5 file.
    
    Args:
        h5_file_path: Path to the predictions HDF5 file
        
    Returns:
        tuple: (predictions, binary_predictions, targets, metadata)
        
    Raises:
        FileNotFoundError: If the predictions file doesn't exist
        Exception: If there's an error loading the file
    """
    if not Path(h5_file_path).exists():
        raise FileNotFoundError(f"Predictions file not found at {h5_file_path}")
    
    try:
        with h5py.File(h5_file_path, 'r') as f:
            predictions = f['predictions'][:]  # type: ignore
            binary_predictions = f['binary_predictions'][:]  # type: ignore
            targets = f['targets'][:]  # type: ignore
            
            # Load metadata
            metadata = {}
            for key in f.attrs.keys():
                metadata[key] = f.attrs[key]
                
            print(f"Loaded predictions from {h5_file_path}")
            print(f"Shape: {predictions.shape}")  # type: ignore
            print(f"Metadata: {metadata}")
            
            return predictions, binary_predictions, targets, metadata
    except Exception as e:
        raise Exception(f"Error loading predictions from {h5_file_path}: {e}")

def get_real_patch(predictions, binary_predictions, targets, patch_idx=None):
    """Get a real patch from loaded predictions.
    
    Args:
        predictions: Array of prediction probabilities
        binary_predictions: Array of binary predictions
        targets: Array of ground truth targets
        patch_idx: Specific patch index to get (if None, random)
        
    Returns:
        tuple: (prediction, binary_prediction, target)
    """
    if patch_idx is None:
        patch_idx = np.random.randint(0, len(predictions))
    
    # Handle different array shapes
    if len(predictions.shape) == 4:  # (batch, channels, height, width)
        prediction = predictions[patch_idx, 0]  # Take first channel
        binary_prediction = binary_predictions[patch_idx, 0]
        target = targets[patch_idx, 0]
    elif len(predictions.shape) == 3:  # (batch, height, width)
        prediction = predictions[patch_idx]
        binary_prediction = binary_predictions[patch_idx]
        target = targets[patch_idx]
    else:
        raise ValueError(f"Unexpected prediction shape: {predictions.shape}")
    
    return prediction, binary_prediction, target

def display_random_patches(num_patches=3):
    """Display multiple random patches from model predictions.
    
    Args:
        num_patches: Number of patches to display
        
    Raises:
        FileNotFoundError: If predictions.h5 file doesn't exist
        Exception: If there's an error loading predictions
    """
    print(f"Displaying {num_patches} random patches...")
    
    # Load predictions from h5 file
    predictions_path = "../results/predictions.h5"
    try:
        predictions, binary_predictions, targets, metadata = load_predictions_from_h5(predictions_path)
        print(f"Successfully loaded predictions from {predictions_path}")
    except FileNotFoundError:
        print(f"\nERROR: Predictions file not found at {predictions_path}")
        print("\n📋 To generate predictions, please follow these steps:")
        print("   1. Train the model: python main.py")
        print("   2. The model will save predictions during evaluation")
        print("   3. Re-run this notebook to visualize the predictions")
        return
    except Exception as e:
        print(f"\nERROR: {e}")
        print("\n📋 Please ensure the predictions file is valid and accessible.")
        return
    
    for i in range(num_patches):
        print(f"\n--- Patch {i+1} ---")
        
        # Get real patch data
        prediction, binary_prediction, target = get_real_patch(
            predictions, binary_predictions, targets, patch_idx=None
        )
        
        # Display statistics
        patch_height, patch_width = prediction.shape  # type: ignore
        total_pixels = patch_height * patch_width
        deforestation_area = np.sum(target) / total_pixels * 100  # type: ignore
        predicted_area = np.sum(binary_prediction) / total_pixels * 100  # type: ignore
        
        print(f"Patch size: {patch_width}x{patch_height}")
        print(f"Actual deforestation area: {deforestation_area:.2f}%")
        print(f"Predicted deforestation area: {predicted_area:.2f}%")
        print(f"Prediction confidence range: [{prediction.min():.3f}, {prediction.max():.3f}]")  # type: ignore
        
        # Calculate accuracy metrics
        # Calculate IoU (Intersection over Union)
        intersection = np.sum((binary_prediction == 1) & (target == 1))
        union = np.sum((binary_prediction == 1) | (target == 1))
        iou = intersection / union if union > 0 else 0
        
        # Calculate accuracy
        accuracy = np.sum(binary_prediction == target) / total_pixels
        
        print(f"Accuracy: {accuracy:.3f}")
        print(f"IoU: {iou:.3f}")
        
        # Plot the patch
        plot_prediction_patch(
            prediction=prediction,
            binary_prediction=binary_prediction,
            target=target,
            patch_idx=i+1,
            figsize=(15, 5),
            show=True
        )

# Set random seed for reproducibility
np.random.seed(42)

# Display random patches from model predictions
display_random_patches(num_patches=4)


Displaying 4 random patches...

ERROR: Predictions file not found at ../results/predictions.h5

📋 To generate predictions, please follow these steps:
   1. Train the model: python main.py
   2. The model will save predictions during evaluation
   3. Re-run this notebook to visualize the predictions
